In [ ]:
import torch
from trl import SFTTrainer
from datasets import load_from_disk
from transformers import TrainingArguments
from utils import process_output
from unsloth import FastLanguageModel, is_bfloat16_supported
import os
import re

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


In [4]:
# Load the dataset
dataset = load_from_disk("../data/fine_tuning_dataset")
# Access train and test splits
train_ds = dataset['train']
test_ds = dataset['test']

In [5]:
max_seq_length = 2048 
dtype = None 
load_in_4bit = False 

In [6]:
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "mistral_fine_tuned",
    max_seq_length = max_seq_length,
    dtype = dtype, 
    load_in_4bit = load_in_4bit,
)
FastLanguageModel.for_inference(model)

==((====))==  Unsloth 2024.9.post4: Fast Mistral patching. Transformers = 4.44.2.
   \\   /|    GPU: NVIDIA GeForce RTX 4070 Laptop GPU. Max memory: 7.996 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.4.1+cu121. CUDA = 8.9. CUDA Toolkit = 12.1.
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.28.post1. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth


model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/155 [00:00<?, ?B/s]

Exception: data did not match any variant of untagged enum ModelWrapper at line 268052 column 3

In [6]:
test_ds['text']
def substitute_last_part_of_prompt(input_string):
    result = re.sub(r'\n{3}"Class": .*?<\|eot_id\|>', '', input_string)
    return result
test_prompts = [substitute_last_part_of_prompt(prompt) for prompt in test_ds['text']]

In [7]:
from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)

In [8]:
predictions = []
# Iterate through all test instances
for i, prompt in enumerate(test_prompts):
    inputs = tokenizer(prompt, return_tensors="pt").to("cuda")
    generated_output = model.generate(**inputs, streamer=text_streamer, max_new_tokens=10)
    predicted_text = tokenizer.decode(generated_output[0], skip_special_tokens=True)
    predictions.append(predicted_text)

<s> <|begin_of_text|><|start_header_id|>system<|end_header_id|>

You are a helpful assistant who is fluent in Russian and can classify text accurately.<|eot_id|><|start_header_id|>user<|end_header_id|>

Your task is to classify Russian text as either "disinformation" or "trustworthy" 
Disinformation is false or misleading information deliberately spread to deceive people. It is often used to manipulate public opinion, create confusion, or influence decisions through inaccurate or fabricated narratives.

```
На радость Гитлеру украинский коп перепутал униформу ЗНАЙ ЮА укр рус укр рус Главная Досье Наше Киев .Знай Важное Жизнь Война Общество Компромат Еще Анекдоты Контакты Реклама Политика конфиденциальности Продолжая просматривать . Вы подтверждаете, что ознакомились с Политикой конфиденциальности и согласны с использованием файлов . Понял Война россии против Украины Коронавирус в Украине и мире Новости дня Главная Общество Читати українською На радость Гитлеру украинский коп перепутал 

In [11]:
processed_outputs = []
for output in predictions:
    result = output.split('\n\nassistant\n\n')[-1]
    processed_outputs.append(process_output(result))

NameError: name 'process_output' is not defined

In [12]:
# Clean true labels 
true_labels = [process_output(label) for label in test_ds['OUTPUT']]

NameError: name 'process_output' is not defined

In [ ]:
from sklearn.metrics import accuracy_score, ConfusionMatrixDisplay, classification_report, recall_score, precision_score, f1_score, confusion_matrix
import matplotlib.pyplot as plt
import numpy as np
# Calculate confusion matrix
cm = confusion_matrix(true_labels, processed_outputs, normalize='true')

# Plot confusion matrix
fig, ax = plt.subplots(figsize=(10, 10))
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=set(processed_true_labels))
disp.plot(ax=ax, cmap='viridis', values_format='.2f')
fig.savefig("cfm.png")

In [ ]:
y_true = processed_true_labels
y_pred = processed_outputs
default_metrics = {
    'accuracy': accuracy_score,
    'recall': lambda y_t, y_p: recall_score(y_t, y_p, zero_division=1, average='macro'),
    'precision': lambda y_t, y_p: precision_score(y_t, y_p, zero_division=1, average='macro'),
    'f1': lambda y_t, y_p: f1_score(y_t, y_p, zero_division=1, average='macro')
}
cls_report = classification_report(y_true, y_pred, output_dict=True)
metrics = {metric_name: metric_func(y_true, y_pred) for metric_name, metric_func in default_metrics.items()}